In [1]:
import re
from typing import NamedTuple
import numpy as np


In [2]:
# 3 tokens for A, 1 token for B
def parse(s):
    groups = s.split('\n\n')
    machines = [re.findall(r'(\d+).*?(\d+)', machine) for machine in groups]
    return np.array( [
        [[int(a), int(b)] for a,b in line]
    for line in machines
    ])


## Part One

Solve the linear equation throwing out any that are not close enough to integers.

In [3]:
def np_solve(v1, v2, B):
    A = np.column_stack((v1, v2))    
    n = np.linalg.solve(A, B)
    
    c_final = n[0] * v1 + n[1] * v2
    
    if np.allclose(n, n.round()):
        n = n.round().astype(int)
        return  3*n[0] + n[1]
    else:
        return 0

In [4]:
sample = '''Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279'''

sample_machines = parse(sample)
sum(np_solve(*machine) for machine in sample_machines)

480

In [5]:
with open('input_files/13.txt') as f:
    raw_inpit = f.read()

machines = parse(raw_inpit)

In [6]:
sum(np_solve(*machine) for machine in machines)

35729

## Part Two

Well…the above won't work for part two because of precision issues with the huge numbers and numerical solutions using np.linalg.solve().

The buttons A and B always have linearly independent vectors. This should simplify things since it means there is one integer solution — it's basically skewed manhattan distance (we don't need to even look for the minimum).


In [7]:
big_machines = machines.copy()
big_machines[:,2] += np.array([10000000000000, 10000000000000])

The plan is to view this as aA + bB = C for buttons A, B, and destination C. We can solve the congruence for a or b and use that to identify the unique solution for b. The input is desgined in a _very_ specific way to allow this. 

See also: https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm



In [8]:
def coef_gcd(a, b):
    '''
    Need a little more than math.gcd gives. To solve equations below we need x and y
    that allows xa + yb = gcd(a,b) where a and b are a column of the button vectors
    '''
    if b == 0:
        return (a, 1, 0)
    else:
        g, x, y = coef_gcd(b, a % b)
        return (g, y, x - (a // b) * y)

def solve_linear_congruence(A, B, C):
    """
    Solves the congruence a * A.row ≡ C.row (mod B.row):
    """
    g, x, y = coef_gcd(A[0], B[0])

    # if c is not divisible by g there's no solution
    if C[0] % g != 0:
        return None  
    
    x *= C[0] // g
    base = B[0] // g
    x = x % base
    return (g, x, base)

def part_two(machine):
    A, B, C = machine
    
    cong_sol = solve_linear_congruence(A, B, C)

    if cong_sol is None: # No solution if the dest is not divisible by gcd a,b
        return None
    
    g, x, base = cong_sol

    # we know the buttons are independent and the determinants
    # will be not be zero. Otherwise there would be infinite (or no) solutions.
    D = A[1] * B[0] - A[0] * B[1]
    E = C[1] * B[0] - C[0] * B[1]

    # We want z where z = (E - x * D) / (base * D)
    # Not sure this is guaranteed to work in general …🤞
    numerator = E - x * D
    denom = base * D
    
    # Solution is only possible if numerator is divisible by denominator 
    if numerator % denom != 0:
        return None

    k = numerator // denom
        
    a = x + k * base
    b = (C[0] - a * A[0]) // B[0]
            
    return 3 * a + b


In [9]:
min_cost = sum(filter(None, (part_two(machine) for machine in big_machines)))

print("Part two:", min_cost)

Part two: 88584689879723
